In [1]:
from copy import deepcopy

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score,\
                            recall_score, roc_auc_score, roc_curve, cohen_kappa_score, fbeta_score
from sklearn.model_selection import GroupKFold


import catboost
from catboost import *

from sklearn.model_selection import train_test_split, KFold, StratifiedKFold

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import warnings
warnings.filterwarnings('ignore')

/opt/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [3]:
samples = pd.read_excel(u'20.01.23 список образцов Моча.xlsx', header=2)
samples.dropna(subset=['Gender'], inplace=True)

samples.fillna(0, inplace=True)
samples.reset_index(drop=True, inplace=True)

In [4]:
spectra1 = pd.read_csv('urine_chm001-250.csv')
spectra2 = pd.read_csv('urine_chm251-500.csv')
spectra_init = pd.concat([spectra1, spectra2],  ignore_index=True)

spectra = spectra_init.filter(regex='^Ch')

mmscaler = MinMaxScaler(feature_range=(-1,1))
# spectra = pd.DataFrame(mmscaler.fit_transform(spectra.T).T)
del(spectra1, spectra2)

frame_lol = samples[['Gender','Age','Dataset']+list(samples.filter(regex=r'_a$').columns)]
spectra['Dataset'] = spectra_init['Dataset']
spectra = pd.merge(left=spectra, right=frame_lol, how='left', on='Dataset')
spectra.dropna(subset=samples.filter(regex=r'_a$').columns, inplace=True)

In [5]:
spectra

,Ch_0_0,Ch_0_1,Ch_0_2,Ch_0_3,Ch_0_4,Ch_0_5,Ch_0_6,Ch_0_7,Ch_0_8,Ch_0_9,Ch_0_10,Ch_0_11,Ch_0_12,Ch_0_13,Ch_0_14,Ch_0_15,Ch_0_16,Ch_0_17,Ch_1_0,Ch_1_1,Ch_1_2,Ch_1_3,Ch_1_4,Ch_1_5,Ch_1_6,Ch_1_7,Ch_1_8,Ch_1_9,Ch_1_10,Ch_1_11,Ch_1_12,Ch_1_13,Ch_1_14,Ch_1_15,Ch_1_16,Ch_1_17,Dataset,Gender,Age,Density_a,pH_a,Protein_a,Bilirubin_a,Glucose_a,Ketones_a,Leukocyte_a,Nitrite_a,Urobilinogen_a,Blood_a,Erythrocyte_a,Squamous cells_a,Hyaline cylinders_a,Bacteria_a,Crystals_a,Ferment_a,Small cells_a,Pathological cylinders_a,Slime_a,Spermatozoon_a,TOTAL_a
0,116.2066,86.67870,501.8073,1966.517,1211.834,3746.119,420.2404,1028.663,1396.969,233.9503,3703.057,52.19898,3828.725,36.83828,5593.055,40.32899,9631.456,42.93369,2088.670,24.48042,1244.148,248.4629,753.4396,126.9792,286.7270,435.5240,241.2221,660.4495,738.9610,748.5693,419.7867,606.7318,113.9607,206.4316,103.15650,146.9493,Chm1,m,24.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,8.0
1,110.4915,89.53624,495.1797,2054.570,1219.735,3993.029,410.4901,1043.288,1414.642,244.8904,3810.526,52.96661,4077.384,38.06622,5931.242,40.90512,9532.379,44.03455,2196.873,25.45964,1191.888,232.8766,775.6388,131.8630,280.0394,426.3287,232.0723,638.8227,729.3528,700.5281,408.5924,614.5677,118.5191,212.2925,106.07600,155.7079,Chm1,m,24.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,8.0
2,103.8239,81.91614,484.7648,1963.676,1213.810,3925.869,400.7397,1040.363,1449.987,248.2566,3932.579,56.03714,4170.093,41.13608,6102.928,44.93802,10287.570,47.33714,2313.889,27.90768,1122.209,232.8766,776.0828,132.3070,288.3988,467.2897,232.0723,661.2813,748.5693,736.3406,400.7564,640.3147,118.5191,227.2702,107.04920,140.1371,Chm1,m,24.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,8.0
3,109.5390,85.72619,510.3286,2041.314,1270.105,4032.534,412.4402,1055.964,1477.758,249.0982,4042.351,56.03714,4242.542,40.52211,6130.583,43.78576,10056.390,46.23627,2243.386,26.43886,1149.714,233.7935,802.2778,137.1908,281.7113,451.4069,231.2405,657.1223,756.4305,726.7324,415.3090,632.4786,121.1239,224.6654,108.02230,163.4933,Chm1,m,24.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,8.0
4,105.7290,82.86865,488.5520,1967.463,1214.797,3926.857,408.5400,1042.313,1459.244,248.2566,3993.222,56.03714,4185.443,40.52211,6128.854,43.78576,10486.830,47.33714,2326.130,27.90768,1142.379,234.7103,777.4147,130.0871,294.2504,482.3366,236.2313,671.2629,765.1653,750.3162,408.5924,574.2682,119.1703,233.7822,107.04920,142.0834,Chm1,m,24.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4869,131.4468,110.49150,632.4666,2789.291,1586.150,5790.534,458.2668,1126.166,1433.998,253.3059,3867.330,59.87530,4161.498,49.11771,5540.051,44.36189,8885.071,48.43800,1967.247,27.90768,1309.243,232.8766,808.4936,142.5186,274.1879,369.4849,231.2405,586.4193,696.1607,647.2460,413.0701,589.9402,114.6119,197.9660,99.26377,140.1371,Chm500,f,65.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,5.0
4870,131.4468,110.49150,633.4134,2793.078,1586.150,5820.163,459.2419,1123.241,1433.998,251.6228,3872.704,60.64293,4162.112,49.11771,5535.442,44.36189,8863.054,47.33714,1963.819,28.39729,1307.410,233.7935,808.9376,140.2987,274.1879,366.9771,230.4087,578.9330,693.5402,660.3481,411.9507,548.5213,114.6119,216.1997,99.26377,130.4053,Chm500,f,65.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,5.0
4871,131.4468,109.53900,634.3602,2764.674,1585.162,5782.632,459.2419,1124.216,1432.315,252.4644,3861.957,59.87530,4145.535,49.11771,5512.397,44.93802,8858.650,48.43800,1955.006,27.90768,1306.493,233.7935,804.0537,143.4066,272.5160,372.8286

In [5]:
@interact
def to_show(selected_target = ['Density_a', 'pH_a', 'Protein_a',
                               'Glucose_a', 'Ketones_a', 'Leukocyte_a', 'Nitrite_a',
                               'Urobilinogen_a', 'Blood_a', 'Erythrocyte_a',
                               'Squamous cells_a', 'Hyaline cylinders_a', 'Bacteria_a',
                               'Crystals_a', 'Ferment_a', 'Small cells_a', 'Pathological cylinders_a',
                               'Slime_a', 'Spermatozoon_a']):
    
    X = spectra.iloc[:,:39]
    y = spectra[selected_target]
    groups = X.Dataset.values
    X.drop('Dataset', axis=1, inplace=True)
    
    gkf = GroupKFold(3)
    
    # counter
    cntr = 0
    plt_flag = False

    for train_indicies, test_indicies in gkf.split(X, y, groups):
        cntr+=1
        if cntr == 3: plt_flag = True

        Train_X, Test_X = X.iloc[train_indicies, :], X.iloc[test_indicies, :]
        Train_y , Test_y = y.iloc[train_indicies], y.iloc[test_indicies]

#         # initialize Pool
#         train_pool = Pool(Train_X, Train_y, cat_features=['Gender'])
#         test_pool = Pool(Test_X, cat_features=[1]) 

        # specify the training parameters 
        model = CatBoostClassifier(
            loss_function='Logloss',
            iterations=5000,
            random_seed=21,
            learning_rate=0.005,
            custom_loss=['AUC', 'Accuracy', 'Precision', 'Recall'],
            eval_metric='Precision',
            use_best_model=True,
            early_stopping_rounds=100,
            class_weights=(0.05, 0.95)
        )
        model.fit(
            Train_X, Train_y,
            cat_features=['Gender'],
            eval_set=(Test_X, Test_y),
            verbose=False,
            plot=plt_flag
        )
        # make the prediction using the resulting model
        preds = model.predict(Test_X)
        print(classification_report(Test_y, preds))
        

interactive(children=(Dropdown(description='selected_target', options=('Density_a', 'pH_a', 'Protein_a', 'Gluc…

## CV for all Anomalies : threshold finding

In [20]:
from tqdm import tqdm_notebook
from itertools import product

In [21]:
def predict(clf, X, trsh):
    probs = clf.predict_proba(X)
    preds = list(map(lambda x: 1 if x>=trsh else 0, probs[:,1]))
    return preds

def metrics_ret(y_true, y_pred, y_prob):
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred)
    rec = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    cohen_kappa = cohen_kappa_score(y_true, y_pred)
    return np.array([acc, prec, rec, f1, cohen_kappa])

In [22]:
Anomaly_Cols = ['Density_a', 'pH_a', 'Protein_a',
               'Glucose_a', 'Ketones_a', 'Leukocyte_a', 'Nitrite_a',
               'Urobilinogen_a', 'Blood_a', 'Erythrocyte_a',
               'Squamous cells_a', 'Hyaline cylinders_a', 'Bacteria_a',
               'Crystals_a', 'Ferment_a', 'Small cells_a', 'Pathological cylinders_a',
               'Slime_a', 'Spermatozoon_a']

Metrics = ['Accuracy', 'Precision', 'Recall', 'F-Score', 'Cohen`s kappa', 'Norm/Anom']

Exam_results_df = pd.DataFrame(index=pd.MultiIndex.from_product([Anomaly_Cols,['CV', 'EXAM']]), columns=Metrics)

In [24]:
## Поиск порога

def max_metric(name, y_true, y_pred):
    if name=='precision':
        return precision_score(y_true, y_pred)
    elif name=='recall':
        return recall_score(y_true, y_pred)
    elif name=='f-score':
        return fbeta_score(y_true, y_pred, beta=1)
    elif name=='roc-auc':
        return roc_auc_score(y_true, y_pred)
    elif name=='cohen':
        return cohen_kappa_score(y_true, y_pred)
    

Best_trsh_regul_dict = {}


X = spectra.iloc[:,:39]

for ANOMAL in tqdm_notebook(Anomaly_Cols):
    Metrics = dict()
    y = spectra[ANOMAL]
    Groups = X.Dataset.values
    X.drop('Dataset', axis=1, inplace=True)


    BEST_TRSH = 0.5
    gkf = GroupKFold(4)
    for TRSH, LR in product(np.arange(0.05,1.0,0.05), [0.08, 0.1, 0.12]):
        temp_results = []
        for train_inds, test_inds in gkf.split(X, y, Groups):
            Train_X, Train_y = X.iloc[train_inds,:], y.iloc[train_inds]
            Test_X, Test_y =  X.iloc[test_inds,:], y.iloc[test_inds]

#             LDA = LinearDiscriminantAnalysis(n_components=1)
#             LDA_model = LDA.fit(Train_X, Train_y)
#             LDA_train = pd.DataFrame(LDA_model.transform(Train_X))
#             LDA_train['target'] = Train_y.values
#             LDA_test = pd.DataFrame(LDA_model.transform(Test_X))
#             LDA_test['target'] = Test_y.values

            model = CatBoostClassifier(
            loss_function='Logloss',
            iterations=500,
            random_seed=21,
            learning_rate=LR,
            custom_loss=['AUC', 'Accuracy', 'Precision', 'Recall'],
            #eval_metric='AUC',
            use_best_model=True,
            early_stopping_rounds=20,
            class_weights=(0.05, 0.95)
            )
    
            model.fit(
                Train_X, Train_y,
                cat_features=['Gender'],
                eval_set=(Test_X, Test_y),
                verbose=False,
                plot=False
            )
            Preds = predict(model, Test_X, TRSH)
            temp_results.append(max_metric('f-score', Test_y, Preds))
            
        Metrics[np.mean(temp_results)] = (TRSH.round(2), LR)
    
    BEST_TRSH_LR = Metrics[max(Metrics)]
    Best_trsh_regul_dict[ANOMAL] = BEST_TRSH_LR

AttributeError: 'DataFrame' object has no attribute 'Dataset'